# Import Library

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

%matplotlib inline
rcParams['figure.figsize'] = 8, 5

# Generate Sin(x) Dataset

In [ ]:
# define input array with angles from 60deg to 300deg converted to radians
x = np.array([i * np.pi / 180 for i in range(60, 300, 4)])
np.random.seed(100)  # Setting seed for reproducability
y = np.sin(x) + np.random.normal(0, 0.15, len(x))

data = pd.DataFrame(np.column_stack([x, y]), columns=['x', 'y'])
plt.plot(data['x'], data['y'], '.')

# Generate New Features with higher power 

In [ ]:
for i in range(2, 16):  # power of 1 is already there
    colname = 'x_%d' % i  # new var will be x_power
    data[colname] = data['x']**i
data.head()

# 0. Function definition

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
#　調整參數造成收斂不完全，省略警告
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def example_regression(data, power, plot_dict, reg_type, alpha, var):
    # define estimator object
    type_dict = {
        'Linear': LinearRegression(),
        'Lasso': Lasso(alpha=alpha),
        'Ridge': Ridge(alpha=alpha)
    }
    if var == 'power':
        # generate X in order
        X = ['x']
        if power >= 2:
            X.extend(['x_%d' % i for i in range(2, power + 1)])

        # fit the model
        if reg_type in type_dict:
            model = type_dict[reg_type]
        model.fit(data[X], data['y'])
        y_pred = model.predict(data[X])

        # check if a plot is to be made for the entered power
        if power in plot_dict:
            plt.subplot(plot_dict[power])
            plt.tight_layout()
            plt.plot(data['x'], data['y'], '.')
            plt.plot(data['x'], y_pred)
            plt.title('Plot for power: %d' % power)

        # return the result in pre-defined format
        rss = sum((y_pred - data['y'])**2)
        ret = [rss]
        ret.extend([model.intercept_])
        ret.extend(model.coef_)
    elif var == 'alpha':
        # generate X in order
        X = ['x']
        if power >= 2:
            X.extend(['x_%d' % i for i in range(2, power + 1)])

        # fit the model
        if reg_type in type_dict:
            model = type_dict[reg_type]
        model.fit(data[X], data['y'])
        y_pred = model.predict(data[X])

        # check if a plot is to be made for the entered power
        if alpha in plot_dict:
            plt.subplot(plot_dict[alpha])
            plt.tight_layout()
            plt.plot(data['x'], data['y'], '.')
            plt.plot(data['x'], y_pred)
            plt.title('Plot for alpha: %.3f' % alpha)

        # return the result in pre-defined format
        rss = sum((y_pred - data['y'])**2)
        ret = [rss]
        ret.extend([model.intercept_])
        ret.extend(model.coef_)
    return ret

# Lasso Regularization with Varied power

In [ ]:
# initialize a dataframe to store the results:
col = ['rss', 'intercept'] + ['coef_x_%d' % i for i in range(1, 16)]
ind = ['power_%d' % i for i in range(1, 16)]

perf_Lasso = pd.DataFrame(index=ind, columns=col)

# define the powers for which a plot is required: {power:where}
plot_dict = {1: 231, 3: 232, 6: 233, 9: 234, 12: 235, 15: 236}

# iterate through all powers and assimilate results
for i in range(1, 16):
    perf_Lasso.iloc[i - 1, 0:i + 2] = example_regression(data,
                                                         power=i,
                                                         plot_dict=plot_dict,
                                                         reg_type='Lasso',
                                                         alpha=1e-3,
                                                         var='power')

In [ ]:
pd.options.display.float_format = '{:,.2g}'.format
perf_Lasso

# Lasso Regularization with Varied Alpha

In [ ]:
# initialize a dataframe to store the results:
col = ['rss', 'intercept'] + ['coef_x_%d' % i for i in range(1, 16)]
ind = ['alpha_%de-3' % i for i in range(1, 16)]

perf_Lasso = pd.DataFrame(index=ind, columns=col)

# define the alpha for which a plot is required: {alpha:where}
plot_dict = {
    1e-3: 231,
    3e-3: 232,
    6e-3: 233,
    7e-3: 234,
    12e-3: 235,
    15e-3: 236
}  # key值為alpha的值

# iterate through all powers and assimilate results
for i in range(1, 16):
    perf_Lasso.iloc[i - 1, 0:18] = example_regression(data,
                                                      power=15,
                                                      plot_dict=plot_dict,
                                                      reg_type='Lasso',
                                                      alpha=i * 1e-3,
                                                      var='alpha')

In [ ]:
pd.options.display.float_format = '{:,.2g}'.format
perf_Lasso

# Ridge Regularization with Varied power

In [ ]:
# initialize a dataframe to store the results:
col = ['rss', 'intercept'] + ['coef_x_%d' % i for i in range(1, 16)]
ind = ['power_%d' % i for i in range(1, 16)]

perf_Ridge = pd.DataFrame(index=ind, columns=col)

# define the powers for which a plot is required: {power:where}
plot_dict = {1: 231, 3: 232, 6: 233, 9: 234, 12: 235, 15: 236}

# iterate through all powers and assimilate results
for i in range(1, 16):
    perf_Ridge.iloc[i - 1, 0:i + 2] = example_regression(data,
                                                         power=i,
                                                         plot_dict=plot_dict,
                                                         reg_type='Ridge',
                                                         alpha=1e-3,
                                                         var='power')

In [ ]:
pd.options.display.float_format = '{:,.2g}'.format
perf_Ridge

# Ridge Regularization with Varied Alpha¶

In [ ]:
# initialize a dataframe to store the results:
col = ['rss', 'intercept'] + ['coef_x_%d' % i for i in range(1, 16)]
ind = ['alpha_%de-3' % i for i in range(1, 16)]

perf_Ridge = pd.DataFrame(index=ind, columns=col)

# define the alpha for which a plot is required: {power:where}
plot_dict = {
    1e-3: 231,
    3e-3: 232,
    6e-3: 233,
    7e-3: 234,
    12e-3: 235,
    15e-3: 236
}  # key值為alpha的值
# iterate through all alphas and assimilate results
for i in range(1, 16):
    perf_Ridge.iloc[i - 1, 0:18] = example_regression(data,
                                                      power=15,
                                                      plot_dict=plot_dict,
                                                      reg_type='Ridge',
                                                      alpha=i * 1e-3,
                                                      var='alpha')

In [ ]:
pd.options.display.float_format = '{:,.2g}'.format
perf_Ridge